In [100]:
import os
import preprocessor as prep
import postprocessor as postp
import processor as proc
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics
import multiprocessing as mp
from multiprocessing import Pool


from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from pandas.api.types import is_numeric_dtype

# 3. Адаптация
Нужно, используя сгенерированные данные `well_name_adapt_input` получить значения коэффициентов калибровок по напору и мощности с помощью `processor.py`

Для этого выключить флаги

* vfm_calc_option = False
* restore_q_liq_only = False

и определить папку с входными данными `dir_name_with_input_data`

In [101]:
dir_name_with_input_data = 'adapt_input_'
well_name = '1628'
path_to_work_dir = f'data/{well_name}/adaptation_/'

Подготовка к параллельному расчету.

В проекте UniflocVBA нужно вручную размножить надстройки до нужного количества потоков - каждая надстройка будет работать параллельно и рассчитывать определенную часть общих данных. 

По умолчанию выставлено 4 потока на 4 надстройках `UniflocVBA_7.xlam`, `UniflocVBA_7_1.xlam`, `UniflocVBA_7_2.xlam`, `UniflocVBA_7_3.xlam`. При желании можно добавить еще, не забыв изменить также номера потоков и их общее количество.

Группировка информации о потоках в единый список `thread_option_list`

Запуск процесса адаптации - получения калибровок. Процесс может быть долгим - в среднем расчет идет медленнее, чем при восстановлении дебитов. 

Рассчет 1 месяца в среднем занимает около 15-20 минут.

Стоит отметить, что для аварийного выхода нужно вручную закрыть надстройки Excel (окна, которые открыты и в них ведется работа). После закрытия всех четырех процесс остановится и возникнет ошибка, контроль над jupyter notebook вернется и можно будет работать.

Поэтому для тщательной отладки `processor.py` рекомендуется запускать отдельно (лучше через PyCharm).

In [102]:
dir_name_with_input_data = 'adapt_input_2019_11_15_14_45_46' + '\\'
input_data_file_name = well_name + '_adapt_input'
calculated_data_file_name = well_name + '_adapt_1'

In [103]:
path_to_work_dir + 'multiprocessing' 

'data/1628/adaptation_/multiprocessing'

In [104]:
filenames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_work_dir + 'multiprocessing\\'):
    filenames_list.extend(filenames)
    break
print(filenames_list)
for i,j in enumerate(filenames_list):
    if i == 0:
        first_result_data = pd.read_csv(path_to_work_dir + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
    else:
        another_result_data = pd.read_csv(path_to_work_dir + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
        first_result_data = first_result_data.append(another_result_data, sort = True)
        del first_result_data['d']
        first_result_data = first_result_data.dropna(subset = ['ESP.ESPpump.EffiencyESP_d'])
first_result_data.to_csv(path_to_work_dir + calculated_data_file_name + '.csv')

['1628_adapt_1.csv', '1628_adapt_2.csv', '1628_adapt_3.csv', '1628_adapt_4.csv', '1628_adapt_5.csv', '1628_adapt_6.csv', '1628_adapt_7.csv', '1628_adapt_8.csv']


Загрузка данных адаптации после расчета - в папке `adaptation_time_mark`  файл `well_name_adapt_1.csv`

In [105]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir +
                                                calculated_data_file_name +  '.csv')

Чтение входных данных для адаптации из папки `adaptation_input` и объединение данных адатации и входных данных для нее.
Сохранение всех данных в папке `adaptation_time_mark` с названием `well_name_calc_and_input.csv`

In [106]:
input_data = pd.read_csv(f'data/{well_name}/adapt_input/' + input_data_file_name +  '.csv')
input_data.index = input_data['Время']
del input_data['Время']
all_data = input_data.join(calculated_data, how = 'outer')

all_data.to_csv(path_to_work_dir + well_name + '_calc_and_input' +  '.csv' )

Построение графиков для данных адаптации в папке  `adaptation_time_mark`

Все данные (входные и результаты) на одном графике `well_name_calc_and_input.html`

In [107]:
adapt_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(adapt_data_traces, plot_file_path, True)

In [108]:
qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)']}
calibrs = {'Калибровки по напору и мощности':
       ['К. калибровки по напору - множитель (Модель)',
           'К. калибровки по мощности - множитель (Модель)']}
metrics = {'Метрики расчета':
       ['Значение функции ошибки (Модель)']}
gor = {'ГФ (Модель)':['ГФ (Модель)']}
wc = {'Обводненность, %':['Обв, % (Модель)']}
pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)', 'P прием ЭЦН, атм (Модель)']}
pressure_wh = {'Рлин': ['Рлин ТМ (Ш)', 'P лин., атм (Модель)']}
pressure_bf = {'Рбуф': ['Рбуф (Ш)', 'P буф., атм (Модель)']}
temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость) (СУ)',
                                                                      'T прием ЭЦН, C (Модель)']}
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)', 'F тока, ГЦ (Модель)']}
choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
true_power = {'Мощность, передаваемая СУ (Модель)':['Мощность, передаваемая СУ (Модель)']}
voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}

efficiency = {'КПД ЭЦН, д.ед.':
          ['КПД ЭЦН, д.ед. (Модель)']}
dif_pressure = {'Перепад давления в ЭЦН, атм':
            ['Перепад давления в ЭЦН, атм (Модель)']}
all_banches = [qliq,calibrs,metrics, gor,wc, pressure_intake, pressure_wh, pressure_bf,
               temp_intake, frequencies, choke,power,true_power, voltage,cos, efficiency, dif_pressure]

pw.create_report_html(all_data, all_banches, path_to_work_dir + 
                      well_name + '_adapt_report.html')